### create a temporary container
```bash
docker run --interactive --tty --rm ubuntu:22.04
docker run -it --rm ubuntu:22.04
```

### create a permanent container

```bash
docker run -it --name my-ubuntu-container ubuntu:22.04
```

### list all the available containers
```bash
docker container ps -a
docker container ps -a | grep my-ubuntu-container # this is a linux command
docker container inspect my-ubuntu-container
```

### to start the container later

```bash
docker start my-ubuntu-container
docker attach my-ubuntu-container
```

### after every new container creation do this
```bash
apt update
```

### to install packages

```bash
apt install iputils-ping --yes
```

### Build a container image with ubuntu image as base and ping installed

```bash
docker build --tag my-ubuntu-image -<<EOF
FROM ubuntu:22.04
RUN apt update && apt install iputils-ping --yes
EOF

# Run a container based on that image
docker run -it --rm my-ubuntu-image

# Confirm that ping was pre-installed
ping google.com -c 1 # Success! 🥳
```

## volume

### 1. volume mount - managed my the docker

```bash
# create a named volume
docker volume create my-volume

# Create a container and mount the volume into the container filesystem
docker run  -it --rm --mount source=my-volume,destination=/my-data/ ubuntu:22.04
# There is a similar (but shorter) syntax using -v which accomplishes the same
docker run  -it --rm -v my-volume:/my-data ubuntu:22.04

# Now we can create and store the file into the location we mounted the volume
echo "Hello from the container!" > /my-data/hello.txt
cat my-data/hello.txt
exit
```

---

```bash
# Create a new container and mount the volume into the container filesystem
docker run  -it --rm --mount source=my-volume,destination=/my-data/ ubuntu:22.04
cat my-data/hello.txt # This time it succeeds! 
exit
```

### 2.bind mount - use the source device storage

```bash
# Create a container that mounts a directory from the host filesystem into the container
docker run  -it --rm --mount type=bind,source="${PWD}"/my-data,destination=/my-data ubuntu:22.04
# Again, there is a similar (but shorter) syntax using -v which accomplishes the same
docker run  -it --rm -v ${PWD}/my-data:/my-data ubuntu:22.04

echo "Hello from the container!" > /my-data/hello.txt

# You should also be able to see the hello.txt file on your host system
cat my-data/hello.txt
exit
```

this is for windows
```bash
docker run -it --rm --mount type=bind,source="%cd%"/my-data,destination=/my-data ubuntu:latest
```

----

<start of the demo app without docker (manual way)>


# Example web app

to manage and automate the task we use makefile

for windows need to install this

```bash
winget install GnuWin32.Make
```
then add the `C:\Program Files (x86)\GnuWin32\bin` path to the environment variables

01. install postgres 

do this on project root dir
```bash
make run-postgres
```

02. install api-node

do this inside the api-node folder

*download nvm for this. its a node version manager.*

```bash
nvm ls
nvm use node 19.4 # for windows nvm use 19.4
npm install
npm run dev # this is done with the make run-api-node
```



3. run api node on terminal

make user the DATABASE_URL is correct (specially the port number)

```bash
make run-api-node
```
if you run this in windows cmd this will note work and it will give an error

so to run this open the project folder in git bash. then run the command (also if you have conda, deactivate it first)

http://localhost:3000/

4. run api-golang

 run these inside of api-golang folder
```bash
mkdir go-workspace
export GOPATH=$PWD/go-workspace
go mod download
go run main.go # use this make run-api-golang
```

on root dir run this 

```bash
make run-api-golang
```

http://localhost:8080/

5. run the client-react

run this on client-react folder

```bash
nvm ls
nvm use node 19.4
npm install
npm run dev
```

<End of the demo app without docker (manual way)>

---

`video time : 1.28.39`

## Dockerfile

first create a file named Dockerfile inside the folder that has your code.

then inside

```Dockerfile
From ubuntu
RUN apt update
RUN apt install nodejs -y 
```
* -y: automatically say yes to the yes no question

and then open the project dir in terminal and run this

```bash
docker build .
```

to add a name for this run build prompt like this

```bash
docker build -t api-node:0 .
```

`always try to reduce the lines in Dockerfile`
```Dockerfile
From ubuntu
RUN apt update && apt install nodejs -y 
```

### always try to use images from docker hub

`this is without using docker hub. it's not perfect`

```Dockerfile
FROM ubuntu

# update the image and install nodejs, npm
RUN apt update && apt install nodejs npm -y

# include our code inside the container
COPY . .

RUN npm install

# to run things on cmd. include the prompt as a list of strings
CMD [ "npm", "run", "dev" ]

```


`with docker hub using the official node image `

```Dockerfile

```



### docker file similar to the production environment



```Dockerfile

FROM node:25-alpine

# ------------------------------------------------------------------------------------------

WORKDIR /usr/src/app
# The command WORKDIR /usr/src/app (note: the standard Linux path is usually /usr, not /user) is used to define the current working directory for any subsequent commands in your Dockerfile.
# ------------------------------------------------------------------------------------------

ENV NODE_ENV=production
#NODE_ENV is the industry-standard variable used to signal whether the application is running in development or production.

#Less Information Leakage: In development, many frameworks show "Stack Traces" (detailed error reports) directly in the browser if your app crashes. In production mode, these are hidden so attackers can't see your code structure.

# Strict Dependencies: When you run npm install, if NODE_ENV is set to production, npm will skip installing devDependencies (like testing tools or linters), keeping your container smaller and more secure.
# ------------------------------------------------------------------------------------------

COPY package*.json ./
# By copying only package.json and running npm install right after, Docker only re-runs the npm install step if package.json or package-lock.json changes. If you only change your application code (after the RUN npm install layer), Docker uses the cached dependency layer, making builds significantly faster.

# ------------------------------------------------------------------------------------------

RUN --mount=type=cache,target=/usr/src/app/.npm \
  npm set cache /usr/src/app/.npm && \
  npm ci --only=production

# combining two powerful concepts: cache mounting and dependency installation

# `npm ci --only=production` the professional way to install dependencies in a production Docker image. It is faster, more secure, and more reliable than the standard npm install.

# ------------------------------------------------------------------------------------------




# include our code inside the container (copy the src folder)
COPY --chown=node:node ./src .
#(using --chown to keep permissions correct)

# ------------------------------------------------------------------------------------------


EXPOSE 3000
#The EXPOSE 3000 instruction is a form of documentation. It tells the person running the container (and the Docker engine) that the application inside is listening on port 3000.

# ------------------------------------------------------------------------------------------


# to run things on cmd. include the prompt as a list of strings
CMD [ "node", "index.js" ] 
# use case - production $ staging
#The primary reason for using CMD ["node", "index.js"] over CMD ["npm", "run", "dev"] or even CMD ["npm", "start"] in a production Docker image is signal handling.
# When running an application inside a container, you want the main process to be able to receive and handle termination signals (like SIGTERM) from the container orchestrator (Docker, Kubernetes, etc.) so it can shut down cleanly.
```

## 📄 Annotated Production Node.js Dockerfile

### 1. Base Image

| Command | Explanation |
| --- | --- |
| **`FROM node:25-alpine`** | Defines the **base image**. You are starting with Node.js version 25 built on **Alpine Linux**. Alpine is chosen for its **extremely small size** and low attack surface, which results in a lightweight, secure container. |

---

### 2. Environment Setup

| Command | Explanation |
| --- | --- |
| **`WORKDIR /usr/src/app`** | Sets the **Current Working Directory**. All subsequent commands (`COPY`, `RUN`, `CMD`) will execute inside this directory. This keeps your application separate from critical system directories. |
| **`ENV NODE_ENV=production`** | Sets the **environment variable** `NODE_ENV`. This is crucial for **optimization and security**: it enables framework caching for performance, suppresses detailed error messages (security), and signals `npm` to skip installing `devDependencies`. |

---

### 3. Dependency Installation Strategy

| Command | Explanation |
| --- | --- |
| **`COPY package.json ./`** | **Caching Layer Optimization.** By copying only `package.json` here (and assuming `package-lock.json` is also copied implicitly or in the next step), Docker creates a build layer that only invalidates if your dependencies change. This makes subsequent builds much faster. |
| **`RUN --mount=type=cache,target=/root/.npm \`** | **BuildKit Cache Mount.** This uses the advanced BuildKit engine to create a persistent, external cache for npm packages. The contents of this cache **are not included** in the final image, keeping it small. |
| **`npm set cache /root/.npm && \`** | Tells the `npm` client running inside the container to store all downloaded packages in the directory mounted by BuildKit. |
| **`npm ci --only=production`** | **Clean, Production Install.** The `ci` (Clean Install) command is faster and more reliable than `npm install` as it strictly adheres to `package-lock.json`. The `--only=production` flag ensures all `devDependencies` are skipped, minimizing final image size. |

---

### 4. Application Code

| Command | Explanation |
| --- | --- |
| **`COPY --chown=node:node ./src .`** | Copies the actual application source code (`./src`) into the working directory (`.`). The **`--chown=node:node`** flag sets the ownership of the files to the non-root user `node`, which is a key security practice. |

---

### 5. Run-Time Configuration

| Command | Explanation |
| --- | --- |
| **`EXPOSE 3000`** | **Documentation.** This instructs Docker that the application inside the container is running and listening on port 3000. It does **not** automatically publish the port to the host machine. |
| **`CMD [ "node", "index.js" ]`** | **Container Entrypoint.** This is the command that runs when the container starts. Using the **JSON array form ("Exec Form")** ensures that the **`node`** process is **PID 1** (the main process). This allows Node.js to receive and handle termination signals (`SIGTERM`) directly from the orchestrator for a clean, graceful shutdown. |

`video time : 1.47.38`